<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_Ensemble_retriever_longContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retriever

**Ensemble = (sparse + Dense) + reorder**

sparse는 키워드를 중심으로 정리하고 검색한다. dense는 문장을 중심으로 정리하고 검색한다. 즉 하나의 키워드라면 sparse가 더 찾을 수 있고, 그걸 어떤 키워드를 문맥적으로 파악하는 것은 dense이다.
그래서 sparse는 이음동의어를 찾을 수 없지만, dense는 이음동의어를 처리할 수 있게 된다.
Ensemble Retriever : 위의 2가지 방식을 써서... 더 정확한 답을 얻을 수 있도록 한다.  


Long context Reorder : 보통 일을 할 때, 도입부에 신경쓰고 마무리에 신경 좀 쓰면 최소노력으로 최대효과를 볼 수 있는데 AI가 똑똑하게도, 주어진 문서 중에 첫 3개정도와 마지막 몇개를 가장 집중적으로 참고한다고 한다(U자 모양) 그래서.. 중요한 문서일 수록 이 앞뒤에 배치해줄 수 있도록 하는 기술.

In [28]:
# colab 환경설정
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
pip install --quiet icecream > /dev/null

In [30]:
import pprint
from icecream import ic

def pp(object):
  ppr = pprint.PrettyPrinter(
      # indent=40,
      width=80
      )
  return ppr.pprint(object)

In [31]:
from IPython.display import clear_output

# Clear all output cells
clear_output()

# Ensenble

In [7]:
pip install -q langchain langchain-openai pypdf sentence-transformers chromadb  faiss-cpu -U rank_bm25 > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.


In [9]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [32]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 1000
TEMP = 1.5
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

openai_compbot = OpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)
openai_chatbot = ChatOpenAI(
    model_name = LLM_MODEL,
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)

ko_embed = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sbert-nli",
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
    )

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

SOURCE_folder = "/content/drive/MyDrive/data/brand"
SOURCE = "/content/source/1-1그로스 해킹, 마케팅과 어떻게 다른가요_ - PUBLY.pdf"

loader = DirectoryLoader(
    SOURCE_folder,
    glob='**/*.pdf',
    show_progress=True,
    loader_cls=PyPDFLoader,
    )

pages = loader.load_and_split()

# 이건 여러번의 로더 호출을 통해, 각 문서가 각각의 로더객체에 담겼을 경우,
# 이 모두를 합해서 하나의 doc 객체로 만들어서 downstream에서 이용하기 위해 합쳐주는 기능.
# 따라서... 우리는 디렉토리 로더로 어차피 하나의 객체에 모든 문서를 '페이지별'로 담았다.
# 이 pypdf는 내부적으로 리컬시브~를 쓴다고 한다.
# docs = []
# for page in pages:
#     docs.append(page.page_content)

ic(len(pages))

In [ ]:
# 이걸 문서별로 덩어리로 만들고 싶다. 즉 불러온 문서가 3개라면, 객체속의 요소도 3개만 존재하도록.
# keys = []
# for page in pages:
#     source = page.metadata['source']
#     if source not in keys:
#         keys.append(page.metadata['source'])
# keys

# 그러나 지금은 일단 포기

lens = []
for page in pages:
    lens.append(len(page.page_content))

print(sorted(lens, reverse = True), end=' ')
print('\n','*'*100, '\n')
print(min(lens), max(lens))

docs = pages

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 적정한 길이로 잘라주고
rc_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=30,
    length_function=len,
)

# chunks = rc_splitter.create_documents(docs) # 옵션을 줄 수 있다. []을 받는다.
chunks = rc_splitter.split_documents(docs) # 옵션을 줄 수 있다. []을 받는다.

len(chunks)

lens = []
for chunk in chunks:
    lens.append(len(chunk.page_content))

print(sorted(lens, reverse = True), end=' ')
print('\n','*'*100, '\n')
print(min(lens), max(lens))

In [ ]:
# for Sparse retriever
bm25_ret = BM25Retriever.from_documents(chunks)
bm25_ret.k=2 # 여기는 선언방식이 진짜 특이하다.

#for Dense retriever
embedding = ko_embed
faiss_db = FAISS.from_documents(chunks, embedding)
faiss_ret = faiss_db.as_retriever(search_kwargs={'k':2})

#init ensemble retriever
ensemble_ret = EnsembleRetriever(
    retrievers = [bm25_ret, faiss_ret], weight=[0.5,0.5]
)

In [ ]:
# 앙상블은 양쪽 리트리버에서 결과를 가져와서
# 순위별로 재정렬도 해준다.

query = '브랜드 네이밍과 인스타그램'

doc = ensemble_ret.invoke(query)

for i in docs:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
docs_faiss = faiss_ret.invoke(query)

for i in docs_faiss:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
docs_bm25 = bm25_ret.invoke(query)

for i in docs_bm25:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = openai_compbot,
    chain_type = 'stuff',
    retriever = faiss_ret,
    return_source_documents = True
)

result_faiss = qa(query)
print(result_faiss['result'])

for i in result_faiss['source_documents']:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = openai_compbot,
    chain_type = 'stuff',
    retriever = bm25_ret,
    return_source_documents = True
)

result_bm25 = qa(query)
print(result_bm25['result'])

for i in result_bm25['source_documents']:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm = openai_compbot,
    chain_type = 'stuff',
    retriever = ensemble_ret,
    return_source_documents = True
)

result = qa(query)
print(result['result'])

for i in result['source_documents']:
    print(i.metadata)
    print("**********************")
    print(i.page_content)
    print("----------------------")

3개의 리트리버를 비교해보면, sparse에서 가장 키워드를 잘 찾아오고(당연한 이야기지만) 문맥적으로는 dense에서 찾는 것을 알 수 있고, 그 순서대로 ensemble에 반영되어 문장이 만들어진다는 것을 확인할 수 있다.

# Long Context Reorder

In [33]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "보스턴 셀틱스에 관한 문서입니다.", "보스턴 셀틱스는 제가 가장 좋아하는 팀입니다.",
    "저는 영화 보러 가는 것을 좋아해요",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적 인 NBA 선수였습니다.",
]

# Create a retriever
retriever = Chroma.from_texts(texts, embedding=ko_embed).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱스에 대해 어떤 이야기를 들려주시겠어요?"

# Get relevant documents ordered by relevance score
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content='보스턴 셀틱스에 관한 문서입니다.'),
 Document(page_content='보스턴 셀틱스에 관한 문서입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='보스턴 셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='보스턴 셀틱스는 제가 가장 좋아하는 팀입니다.')]

In [34]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

# 순서를 잘 살펴보면... 질문과 관련된 문서인 '셀틱스'관련 문서들은 앞과 뒤로 밀어 넣은 것을 알 수 있다.
# 그리고 그런지 아닌지는 잘 모르겠지만, 여튼...'기분'과 관련된 자료를 가장 먼저 배치했음을 알 수 있다.

[Document(page_content='보스턴 셀틱스에 관한 문서입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='보스턴 셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='보스턴 셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='보스턴 셀틱스에 관한 문서입니다.')]

In [35]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
import os
os.environ["OPENAI_API_KEY"] = 'YOUR_API_KEY'

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

template = """Given this text extracts:
-----
{context}
-----
Please answer the following question:
{query}"""
prompt = PromptTemplate(
    template=template, input_variables=["context", "query"]
)
# openai = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0)

# 여기서는 '커스텀'체인을 구성해줘야 한다고 한다.
llm_chain = LLMChain(llm=openai_chatbot, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context" # 컨텍스트라는 이름으로 돌아오는 애가 바로 저 위에 context라는 걸 알려주는 문구
)


In [36]:
reordered_result = chain.run(input_documents=reordered_docs, query=query)
result = chain.run(input_documents=docs, query=query)

print(reordered_result)
print("-"*100)
print(result)

셀틱스는 제작자가 가장 좋아하는 팀입니다. L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.
----------------------------------------------------------------------------------------------------
셀틱스는 제가 가장 좋아하는 팀입니다. L. 코넷은 최고의 셀틱스 선수 중 한 명입니다. 보스턴 셀틱스는 제가 가장 좋아하는 팀입니다.
